In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

## Define Arguments

In [2]:
# Define parser arguments
# nz: size of latent variable
parser = {
    "dataset": "cifar10",
    "dataroot": "./",
    "workers": 2,
    "batchSize": 64,
    "imageSize": 32,
    "nz": 100,
    "ngf": 64,
    "ndf": 64,
    "epochs": 7,
    "lr": 0.0002,
    "beta1": 0.5,
    "cuda": False,
    "ngpu": 1,
    "netG": '',
    "netD": '',
    "outf": './output',
    "manualSeed": 7
}

In [3]:
# Parse arguments for  model
args = argparse.Namespace(**parser) # parse arguments

In [4]:
# Create output
try:
    os.makedirs(args.outf)
except OSError:
    pass

In [5]:
if args.manualSeed is None:
    args.manualSeed = random.randint(1, 10000)
print("Random Seed: ", args.manualSeed)
random.seed(args.manualSeed)
torch.manual_seed(args.manualSeed)
if args.cuda:
    torch.cuda.manual_seed_all(args.manualSeed)

Random Seed:  7


In [6]:
if torch.cuda.is_available() and not opt.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

## Prepare Data

In [7]:
dataset = dset.CIFAR10(root=args.dataroot, download=False,
                           transform=transforms.Compose([
                               transforms.Scale(args.imageSize),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

In [8]:
# DataLoader 
assert dataset
dataloader = torch.utils.data.DataLoader(dataset, batch_size=args.batchSize,shuffle=True, num_workers=int(args.workers))

In [9]:
ngpu = int(args.ngpu)
nz = int(args.nz) #     z_dim
ngf = int(args.ngf) # g_dim??
ndf = int(args.ndf) # d_dim??
nc = 3 # Image channels R, G, B

In [10]:
# custom weights initialization called on netG and netD
# m: model
# Init weights and bias for G and D
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

## Define G


In [11]:
class _netG(nn.Module):
    def __init__(self):
        super(_netG, self).__init__()
        #self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     nz, ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 2, ngf * 1, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 1),
            nn.ReLU(True),

            # state size. (ngf) x 16 x 16
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 32 x 32
        )

    def forward(self, input):
        #gpu_ids = None
        #if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
        #    gpu_ids = range(self.ngpu)
        #return nn.parallel.data_parallel(self.main, input, gpu_ids)
        return self.main(input)

### Init weight G

In [12]:
netG = _netG()
netG.apply(weights_init)
#if opt.netG != '':
#    netG.load_state_dict(torch.load(opt.netG))
print(netG)

_netG (
  (main): Sequential (
    (0): ConvTranspose2d(100, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU (inplace)
    (9): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): Tanh ()
  )
)


## Define D

In [13]:
class _netD(nn.Module):
    def __init__(self):
        super(_netD, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 32 x 32
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf) x 16 x 16
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*2) x 8 x 8
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        #gpu_ids = None
        #if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
        #    gpu_ids = range(self.ngpu)
        #output = nn.parallel.data_parallel(self.main, input, gpu_ids)
        #return output.view(-1, 1)
        return self.main(input)

In [14]:
netD = _netD()
netD.apply(weights_init)
print(netD)

_netD (
  (main): Sequential (
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU (0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU (0.2, inplace)
    (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (9): Sigmoid ()
  )
)


In [15]:
criterion = nn.BCELoss()

In [16]:
#input = torch.FloatTensor(args.batchSize, 3, args.imageSize, args.imageSize)
#noise = torch.FloatTensor(args.batchSize, nz, 1, 1)
#fixed_noise = torch.FloatTensor(args.batchSize, nz, 1, 1).normal_(0, 1)
#label = torch.FloatTensor(args.batchSize)
#real_label = 1
#fake_label = 0

In [17]:
#input = Variable(input)
#label = Variable(label)
#noise = Variable(noise)
#fixed_noise = Variable(fixed_noise)

# setup optimizer
D_optimizer = optim.Adam(netD.parameters(), lr=args.lr, betas=(args.beta1, 0.999))
G_optimizer = optim.Adam(netG.parameters(), lr=args.lr, betas=(args.beta1, 0.999))

In [18]:
def train(epoch):
    for i, (data, _) in enumerate(dataloader):
        # Update net D
        ## 1. Train with real 
        real_label = Variable(torch.ones(data.size(0)))
        fake_label = Variable(torch.zeros(data.size(0)))
        
        data_input = Variable(data)
        D_optimizer.zero_grad()
        
        D_real_output = netD(data_input)
        errD_real = criterion(D_real_output, real_label)
        errD_real.backward()
        D_x = D_real_output.data.mean()
        
        ## 2. Train with fake
        z = Variable(torch.randn(data.size(0), nz, 1, 1))
        G_fake_output = netG(z)
        D_fake_output = netD(G_fake_output.detach())
        errD_fake = criterion(D_fake_output, fake_label)
        errD_fake.backward()
        D_G_z1 = D_fake_output.data.mean()
        errD = errD_real + errD_fake
        D_optimizer.step()
        
        # Update net G
        netG.zero_grad()
        
        D_fake_output_for_G = netD(G_fake_output)
        errG = criterion(D_fake_output_for_G, real_label)
        errG.backward()
        D_G_z2 = D_fake_output_for_G.data.mean()
        G_optimizer.step()
        
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, args.epochs, i, len(dataloader),
                 errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            vutils.save_image(data, 
                              '%s/real_samples.png' % args.outf)
            fake = netG(z)
            vutils.save_image(fake.data,
                              '%s/fake_samples_epoch_%03d.png' % (args.outf, epoch))

In [19]:
for epoch in range(args.epochs):
    train(epoch)

[0/7][0/782] Loss_D: 2.2511 Loss_G: 1.9060 D(x): 0.4320 D(G(z)): 0.6970 / 0.1812
[0/7][1/782] Loss_D: 1.6067 Loss_G: 2.3086 D(x): 0.4786 D(G(z)): 0.5104 / 0.1124
[0/7][2/782] Loss_D: 1.1202 Loss_G: 2.5234 D(x): 0.5530 D(G(z)): 0.3640 / 0.0991
[0/7][3/782] Loss_D: 0.9129 Loss_G: 2.5712 D(x): 0.6553 D(G(z)): 0.3371 / 0.0897
[0/7][4/782] Loss_D: 0.8061 Loss_G: 2.7044 D(x): 0.7424 D(G(z)): 0.3663 / 0.0771
[0/7][5/782] Loss_D: 0.9128 Loss_G: 2.8198 D(x): 0.6902 D(G(z)): 0.3543 / 0.0712
[0/7][6/782] Loss_D: 0.8338 Loss_G: 2.9883 D(x): 0.7274 D(G(z)): 0.3636 / 0.0620
[0/7][7/782] Loss_D: 0.8118 Loss_G: 3.2244 D(x): 0.7040 D(G(z)): 0.3304 / 0.0451
[0/7][8/782] Loss_D: 0.8056 Loss_G: 3.3952 D(x): 0.7524 D(G(z)): 0.3710 / 0.0413
[0/7][9/782] Loss_D: 0.7459 Loss_G: 3.6645 D(x): 0.7513 D(G(z)): 0.3313 / 0.0315
[0/7][10/782] Loss_D: 0.8713 Loss_G: 3.7833 D(x): 0.7385 D(G(z)): 0.3998 / 0.0271
[0/7][11/782] Loss_D: 1.0306 Loss_G: 4.0137 D(x): 0.6951 D(G(z)): 0.4360 / 0.0224
[0/7][12/782] Loss_D: 0.83

Process Process-1:
Process Process-2:
Traceback (most recent call last):
  File "/home/hminle/anaconda2/envs/pydata35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/hminle/anaconda2/envs/pydata35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/hminle/anaconda2/envs/pydata35/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hminle/anaconda2/envs/pydata35/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hminle/anaconda2/envs/pydata35/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 26, in _worker_loop
    r = index_queue.get()
  File "/home/hminle/anaconda2/envs/pydata35/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 26, in _worker_loop
    r = index_queue.get()
  File "/home/hminle/ana

KeyboardInterrupt: 